## Module_2: *(Template)*

## Team Members:
*Trey, Karinna*

## Project Title:
*Quantifiying and Predicting Lung Fibrosis Across Tissue Depths*



## Project Goal:
This project seeks to determine how the extent of lung fibrosis changes with depth within a fibrotic lung and to devlop an image analysis method that quantifies fibrosis across lung biopsy images.

## Disease Background: 
*Fill in information and please note that this module is truncated and only has 5 bullets (instead of the 11 that you did for Module #1).*

* Prevalence & incidence
    - Annual incidence of IPF in the USA is estimated at 6.8 to 8.8 per 100,000 and up to 16.3 to 17.4 per 100,0000 with a broader definition https://pmc.ncbi.nlm.nih.gov/articles/PMC9487229
    - Incidence rate of IPF was 13.2-26.1/100,000 
    https://journal.chestnet.org/article/S0012-3692(24)02766-1/fulltext
* Risk factors (genetic, lifestyle)
    - Approximately 10-20% of IPF cases are familial
    - There are associated variants in genes like MUC5B, TERT, TERC, surfactant genes, and telomere maintaince genes
    https://emedicine.medscape.com/article/301226-overview
    - Incidence of IPF increases dramatically with age, typically around 50-70 years
    - Men are affected more frequently
    - mdpi.com/1422-0067/22/11/6107
    - Smoking, chronic dustinhalation, and occupational exposures increase risk
    https://www.cdc.gov/mmwr/volumes/74/wr/mm7407a1.htm?
* Symptoms
    -shortness of breath
    - dry cough
    - extreme tiredness
    - unintentional weight loss
    - aching muscles and joints
    - clubbing of toes and fingers
    https://www.mayoclinic.org/diseases-conditions/pulmonary-fibrosis/symptoms-causes/syc-20353690
* Standard of care treatment(s)
* Biological mechanisms (anatomy, organ physiology, cell & molecular physiology)

## Data-Set: 
*The dataset analyzed in this project consists of 78 graysclae histological images of bleomycin-induced fibrotic mouse lungs, each captured at different tissue depths ranging from 0-10,000 micrometers. These data were collected by administering bleomycin, a chemotherapeutic antibiotic known to cause IPF, to mice and subsequently, harvesting their lung tissue. The tissue was then sectioned and immunostained for proteins relevant to fibrosis, including desmin, alpha-smooth muscle actin, and CD-31. These stained sections were then imaged using high-resolution Aperio AT2 whole-slide imaging, producing graysclae images in which white regions correspond to fibrotic regions and black regions represent healthy lung tissues.  which investigates molecular


(Describe the data set(s) you will analyze. Cite the source(s) of the data. Describe how the data was collected -- What techniques were used? What units are the data measured in? Etc.)*

## Data Analyis: 
*(Describe how you analyzed the data. This is where you should intersperse your Python code so that anyone reading this can run your code to perform the analysis that you did, generate your figures, etc.)*

## Verify and validate your analysis: 
*(Describe how you checked to see that your analysis gave you an answer that you believe (verify). Describe how your determined if your analysis gave you an answer that is supported by other evidence (e.g., a published paper).*

## Conclusions and Ethical Implications: 
*(Think about the answer your analysis generated, draw conclusions related to your overarching question, and discuss the ethical implications of your conclusions.*

## Limitations and Future Work: 
*(Think about the answer your analysis generated, draw conclusions related to your overarching question, and discuss the ethical implications of your conclusions.*

## NOTES FROM YOUR TEAM: 
*This is where our team is taking notes and recording activity.*

## QUESTIONS FOR YOUR TA: 
Not sure about sources for the data-set